# Detección por Webcam usando 'Harr Cascades' y Filtros de Detección de Borde (Canny)

En este documento, utilizaremos OpenCV que es una librería de vision por computador para tomar datos de una webcam y:
- Detectar Caras en "Tiempo Real" usando "Haar Cascades"
- Detectar Bordes en "Tiempo Real" usando el filtro "Canny"

Referencias:

https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml

### Paso 1: Cargar el Overlay

Se importan el overlay basico de PYNQ al FPGA

In [1]:
from pynq import Overlay
Overlay("base.bit").download()

### Paso 2: Inicializa los drivers de WebCam y HDMI

Importamos los drivers de HDMI de la tarjeta y configuramos la salida HDMI para utilizar y desplegar

In [9]:
# Configuración de video de 640*480 @ 60Hz
from pynq.drivers import HDMI 
#from pynq.drivers.video import VMODE_640x480
#hdmi_out = HDMI('out', video_mode=VMODE_640x480)
from pynq.drivers.video import VMODE_1280x720
hdmi_out = HDMI('out', video_mode=VMODE_1280x720)
hdmi_out.start()

### Paso 3: Declaramos algunas constantes

Constantes de salida para el monitor y la webcam

In [10]:
# Salida del frame buffer del monitor
frame_out_w = 1920
frame_out_h = 1080
# Configuración de la cámara de salida
frame_in_w = 1280
frame_in_h = 720

### Paso 4:  Llamado de librerías

Llamamos el overlay driver de trama como utilitario de las lecturas realizadas por la cámara.
Importamos las librerías de OpenCV igualmente.  
Hacemos llamado también a la librería numpy que nos ayudará a procesar arreglos y vectores.  
Finalmente llamamos a la librería de botones que nos ayudará a poder interrumpir el código para saltar a las siguientes demostraciones.

In [11]:
# Llamados a las librerias a utilizar
from pynq.drivers import Frame
import cv2
import numpy as np
from pynq.board import Button

### Paso 5:  Inicialización de cámara

Abrimos una captura de video con las propiedades de las constantes declaradas anteriormente para procesar cada trama.  
Cabe destacar que a una menor resolución el sistema se desempeñara de manera más rápida porque no tendrá muchos pixeles que procesar.  
Si el driver de la cámara se ejecuta con éxito tendremos acceso a las tramas de la cámara.

In [12]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


### Paso 6: Uso de Haar Cascades

Cargamos los modelos de vision para haarcascades que se encuentran dentro de nuestra tarjeta sd en esta.  
Seguidamente pasamos los pixeles de la cámara a la pantalla y desplegamos en el monitor.  
Aplicamos 'haar cascades' para detectar caras.  
A las caras las escogemos como región de interes para luego detectar ojos.  
Finalmente ponemos cajas para cada caso.  
Desplegamos por HDMI.

In [13]:
# Carga los modelos de Haar desde la tarjeta SD
face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./data/haarcascade_eye.xml')

# Repite indefinidamente
while (True):
    ret, frame_webcam = videoIn.read() # captura una trama de video

    # Despliega en la salida HDMI a todo color
    if (ret):
        frame_1080p = np.zeros((frame_out_h,frame_out_w,3)).astype(np.uint8)                  # la trama de 1080p
        frame_1080p[0:frame_in_h,0:frame_in_w,:] = frame_webcam[0:frame_in_h,0:frame_in_w,:]  # match de la trama de webcam
        hdmi_out.frame_raw(bytearray(frame_1080p.astype(np.int8).tobytes()))                  # desplegamos la salida
    else:
        raise RuntimeError("Failed to read from camera.")                                     # error de lectura
    np_frame = frame_webcam
    
    # cambiamos a modo de escala de grises y ajustamos los parámetros de deteccion en Haar
    gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # buscamos los 'bounding boxes' o cajas donde se encuentran las caras
    for (x,y,w,h) in faces:
        cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2) # dibujamos un rectángulo en la cara
        roi_gray = gray[y:y+h, x:x+w]                       # tomamos una parte de la imagen a B&W
        roi_color = np_frame[y:y+h, x:x+w]                  # tomamos una parte de la imagen a color

        eyes = eye_cascade.detectMultiScale(roi_gray)       # procedemos a detectar los ojos
        for (ex,ey,ew,eh) in eyes:                          # a las imagenes de grises anteriores, detectamos ojos
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2) # ubicamos un rectángulo
    # Sacamos el resultado por HDMI
    frame_1080p[0:frame_in_h,0:frame_in_w,:] = frame_webcam[0:frame_in_h,0:frame_in_w,:] # trama a 1080p a procesar
    hdmi_out.frame_raw(bytearray(frame_1080p.astype(np.int8).tobytes()))  # convertimos la trama a bytes
    if (Button(0).read()==1):                               # si detectamos el botón BTN0
        break                                               # rompemos
    del ret, frame_webcam, frame_1080p, np_frame, gray      # borramos variables temporales para evitar volcado de pila

### Paso 7: Uso de Canny

El filtro de Canny, nombre que obtiene por su creador, es uno de los filtros más conocidos para detección de bordes.  
Los SDC o automoviles autónomos usan un filtro similar para detectar líneas de la calle.  
Aquí solo la utilizaremos para detectar bordes

In [14]:
frame_1080p = np.zeros((frame_out_h,frame_out_w,3)).astype(np.uint8)
while (True):
    # Lee la siguiente imagen
    ret, frame_webcam = videoIn.read()                  # captura una trama
    if (ret):
        frame_canny = cv2.Canny(frame_webcam,100,110)   # aplica el filtro canny
        
        for i in range(3):                              # copia a los tres canales 
            frame_1080p[0:frame_in_h,0:frame_in_w,i] = frame_canny[0:frame_in_h,0:frame_in_w]

                                                        # Copia el frame buffer y lo despliega en el monitor
        hdmi_out.frame_raw(bytearray(frame_1080p.astype(np.int8).tobytes()))
    if (Button(0).read()==1):                           # Termina la ejecución si el botón 0 es presionado
        break
    del ret, frame_webcam

### Paso 8: Liberamos los recursos de HDMI y cámara

Finalmente liberamos el HDMI y la cámara de la memoria.

In [15]:
videoIn.release()       # retiramos el driver de la webcam
hdmi_out.stop()         # retiramos el driver de hdmi
del hdmi_out, videoIn   # borramos las variables temporales